<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interfaces: From Protocols to ABCs

In [0]:
# Example 11-1.  x and y are public data attributes 

class Vector2d:
  typecode = 'd'
  
  def __init__(self, x, y):
    self.x = float(x)
    self.y = float(y)
    
  def __iter__(self):
    return (i for i in (self.x, self.y))

In [0]:
v = Vector2d(3, 4)
v.x

3.0

### Remark

In Chapter 9, we turned x and y into read-only properties (Example 11-2). This is a significant refactoring, but an essential part of the interface of Vector2d is unchanged:
users can still read my_vector.x and my_vector.y.

In [0]:
# Example 11-2. x and y reimplemented as properties

class Vector2d:
  typecode = 'd'
  
  def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)
  
  @property
  def x(self):
    return self.__x
  
  @property
  def y(self):
    return self.__y
  
  def __iter__(self):
    return (i for i in (self.x, self.y))
  
  

In [0]:
v = Vector2d(6, 7)
v.x

6.0

### What is interface?

A useful complementary definition of __interface__ is: the subset of an object’s public methods that enable it to play a specific role in the system. That’s what is implied when the
Python documentation mentions “a file-like object” or “an iterable,” without specifying
a class. An interface seen as a set of methods to fulfill a __role__ is what Smalltalkers called
a __procotol__, and the term spread to other dynamic language communities. __Protocols are
independent of inheritance.__ A class may implement several protocols, enabling its in‐
stances to fulfill several roles.

Protocols are interfaces, but because they are informal—defined only by documentation
and conventions—protocols cannot be enforced like formal interfaces can (we’ll see
how ABCs enforce interface conformance later in this chapter). A protocol may be
partially implemented in a particular class, and that’s OK. Sometimes all a specific API
requires from “a file-like object” is that it has a .read() method that returns bytes. The
remaining file methods may or may not be relevant in the context.

### Remark

that “X-like object,” “X protocol,” and “X interface” are synonyms in the
minds of Pythonistas

In [0]:
# Example 11-3. Partial sequence protocol implementation with __getitem__: enough for item access, iteration, and the in operator

class Foo:
  def __getitem__(self, pos):
    return range(0, 30, 10)[pos]

In [0]:
f = Foo()
f[0]

0

### Remark

There is no method \_\_iter\_\_ yet Foo instances are iterable because—as a fallback—
when Python sees a \_\_getitem\_\_ method, it tries to iterate over the object by calling
that method with integer indexes starting with 0. Because Python is smart enough to
iterate over Foo instances, it can also make the in operator work even if Foo has no
\_\_contains\_\_ method: it does a full scan to check if an item is present.

__summary__:

given the importance of the sequence protocol, in the absence \_\_iter\_\_
and \_\_contains\_\_ Python still manages to make iteration and the in operator work by
invoking \_\_getitem\_\_.

In [0]:
# Example 11-4. A deck as a sequence of cards (same as Example 1-1)

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQAK')
  suits =  'spades diamonds clubs hearts'.split()
  
  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits 
                                    for rank in self.ranks]

  def __len__(self):
    return len(self._cards)
  
  def __getitem__(self, position):
    return self._cards[position]

In [0]:
from random import shuffle
>>> deck = FrenchDeck()
>>> shuffle(deck)

TypeError: ignored

### Remark

The error message is quite clear:__ “'FrenchDeck' object does not support item assignment.”__ The problem is that shuffle operates by swapping items inside the collection,
and FrenchDeck only implements the immutable sequence protocol. Mutable sequences
must also provide a \_\_setitem\__ method.


In [0]:
def set_card(deck, position, card):
  deck._cards[position] = card
    
#Assign that function to an attribute named __setitem__ in the FrenchDeck class.    
FrenchDeck.__setitem__ = set_card

# deck can now be sorted because FrenchDeck now implements the necessary
# method of the mutable sequence protocol.
shuffle(deck)
deck[:5]

[Card(rank='3', suit='spades'),
 Card(rank='J', suit='diamonds'),
 Card(rank='4', suit='hearts'),
 Card(rank='A', suit='spades'),
 Card(rank='5', suit='clubs')]

### Monkey Patching
The trick is that set_card knows that the deck object has an attribute named _cards,
and _cards must be a mutable sequence. The set_card function is then attached to the
FrenchDeck class as the \_\_setitem\____ special method. 

This is an example of __monkey
patching__: changing a class or module at runtime, without touching the source code.
Monkey patching is powerful, but the code that does the actual patching is very tightly
coupled with the program to be patched, often handling private and undocumented
parts.

### Remark

However, even with ABCs, you should beware that excessive use of isinstance checks
may be a code smell—a symptom of bad OO design. It’s usually not OK to have a chain
of if/elif/elif with insinstance checks performing different actions depending on
the type of an object: you should be using polymorphism for that—i.e., designing your
classes so that the interpreter dispatches calls to the proper methods, instead of you
hardcoding the dispatch logic in if/elif/elif blocks.

In [0]:
# Example 11-7. Duck typing to handle a string or an iterable of strings
try:
  field_names = field_names.replace(',', ' ').split()
except AttributeError:
  pass
field_names = tuple(field_names)


## Subclassing an ABC

In [0]:
# Example 11-8. frenchdeck2.py: FrenchDeck2, a subclass of 
# collections.MutableSequence

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQAK')
  suits =  'spades diamonds clubs hearts'.split()
  
  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits 
                                    for rank in self.ranks]

  def __len__(self):
    return len(self._cards)
  
  def __getitem__(self, position):
    return self._cards[position]
  
  def __setitem__(self, position, value):
    # __setitem__ is all we need to enable shuffling…
    self._cards[position] = value
    
  def __delitem__(self, position):
    # But subclassing MutableSequence forces us to implement __delitem__, an
    # abstract method of that ABC
    def self._cards[position]
    
  def insert(self, position, value):
    # We are also required to implement insert, the third abstract method of
    # MutableSequence.
    self._cards.insert(position, value)
  

### Remark: Important

Python does not check for the implementation of the abstract methods at import time
(when the frenchdeck2.py module is loaded and compiled), but only at runtime when
we actually try to instantiate FrenchDeck2. Then, __if we fail to implement any abstract
method, we get a TypeError exception with a message such as "Can't instantiate abstract class FrenchDeck2 with abstract methods \_\_delitem\_\_, insert".
That’s why we must implement \_\_delitem\_\_ and insert, even if our FrenchDeck2
examples do not need those behaviors: the MutableSequence ABC demands them.__

Remark:
Source: https://www.quora.com/What-is-an-abstract-method-in-Python 

if you define a class using abc, creating an instance of a derived class will raise an exception if the derived class doesn’t contain implementations of all of the abstract methods.

## ABCs in the Standard Library

#### Iterable, Container, and Sized


Every collection should either inherit from these ABCs or at least implement com‐
patible protocols. Iterable supports iteration with __iter__, Container supports
the in operator with __contains__, and Sized supports len() with __len__.


#### Sequence, Mapping, and Set


These are the main immutable collection types, and each has a mutable subclass. A
detailed diagram for MutableSequence is in Figure 11-2; for MutableMapping and
MutableSet, there are diagrams in Chapter 3 (Figures 3-1 and 3-2).


#### MappingView


In Python 3, the objects returned from the mapping methods .items(), .keys(),
and .values() inherit from ItemsView, ValuesView, and ValuesView, respectively.
The first two also inherit the rich interface of Set, with all the operators we saw in
“Set Operations” on page 82.


#### Callable and Hashable


These ABCs are not so closely related to collections, but collections.abc was the
first package to define ABCs in the standard library, and these two were deemed
important enough to be included. I’ve never seen subclasses of either Callable or
Hashable. Their main use is to support the insinstance built-in as a safe way of
determining whether an object is callable or hashable.7


#### Iterator


Note that iterator subclasses Iterable. We discuss this further in Chapter 14

## Defining and Using an ABC

In [0]:
# Example 11-9 shows the definition of the Tombola ABC.
# Tombola is an ABC with two abstract methods and two concrete methods

import abc
# To define an ABC, subclass abc.ABC.
class Tombola(abc.ABC):
  
  # An abstract method is marked with the @abstractmethod decorator, and often
  # its body is empty except for a docstring.
  @abc.abstractmethod
  def load(self, iterable):
    """Add items from an iterable"""
    
  @abc.abstractmethod
  def pick(self):
    """Remove item at random, returning it.
       This method should raise `LookupError` when the instance is empty.
    """
    
  def loaded(self):
    """Return `True` if there's at least 1 item, `False` otherwise."""
    # Remark: Concrete methods in an ABC must rely only on the interface defined
    # by the ABC (i.e., other concrete or abstract methods or properties of the 
    # ABC).
    return bool(self.inspect())
  
  def inspect(self): # TO BE REVIEWD
    """Return a sorted tuple with the items currently inside."""
    items = []
    # We can’t know how concrete subclasses will store the items, but we can 
    # build the inspect result by emptying the Tombola with successive calls to 
    # .pick()…
    while True:
      try:
        items.append(self.pick())
      except LookupError:
        break
    # …then use .load(…) to put everything back
    self.load(items)
    return tuple(sorted(items))

### Remark

An abstract method can actually have an implementation. Even if
it does, subclasses will still be forced to override it, but they will be
able to invoke the abstract method with super(), adding func‐
tionality to it instead of implementing from scratch.

### Remark
The .inspect() method in Example 11-9 is perhaps a silly example, but it shows that,
given .pick() and .load(…) we can inspect what’s inside the Tombola by picking all
items and loading them back. The point of this example is to highlight that it’s OK to
provide concrete methods in ABCs, as long as they only depend on other methods in
the interface. Being aware of their internal data structures, concrete subclasses of Tombola may always override .inspect() with a smarter implementation, but they don’t have to.

The .loaded() method in Example 11-9 may not be as silly, but it’s expensive: it
calls .inspect() to build the sorted tuple just to apply bool() on it. This works, but a
concrete subclass can do much better, as we’ll see.

In [0]:
# Example 11-11. A fake Tombola doesn’t go undetected
class Fake(Tombola): #
  def pick(self):
     return 13

Fake 

__main__.Fake

In [0]:
f = Fake()

TypeError: ignored

### interesting

TypeError is raised when we try to instantiate Fake. The message is very clear:
Fake is considered abstract because it failed to implement load, one of the
abstract methods declared in the Tombola ABC.

In [0]:
# Example 11-12. BingoCage is a concrete subclass of Tombola

import random

class BingoCage(Tombola):
  
  def __init__(self, items):
    self._randomizer = random.SystemRandom()
    self._items = []
    
    # Delegate initial loading to the .load(…) method
    self.load(items)
    
  def load(self, items):
    self._items.extend(items)
    
    # Instead of the plain random.shuffle() function, we use the .shuffle() method
    # of our SystemRandom instance.
    self._randomizer.shuffle(self._items)
    
  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty BingoCage')
      
  def __cal__(self):
    
    # __call__ is not needed to satisfy the Tombola
    # interface, but there’s no harm in adding extra methods.
    self.pick()

### Remark

__BingoCage__ inherits the expensive loaded and the silly inspect methods from Tombola. Both could be overridden with much faster one-liners, as in Example 11-13. The
point is: we can be lazy and just inherit the suboptimal concrete methods from an ABC.
The methods inherited from Tombola are not as fast as they could be for BingoCage, but
they do provide correct results for any Tombola subclass that correctly implements pick
and load

In [0]:
# Example 11-13.LotteryBlower is a concrete subclass that overrides the inspect
 # and loaded methods from Tombola
  
import random

class LotteryBlower(Tombola):
  
  def __init__(self, iterable):
    
    # The initializer accepts any iterable: the argument is used to build a list.
    self._balls = list(iterable)
    
  def load(self, iterable):
    self._balls.extend(iterable)
    
  def pick(self):
    
    # The random.randrange(…) function raises ValueError if the range is empty, 
    # so we catch that and throw LookupError instead, to be compatible with 
    # Tombola.
    try:
      position = random.randrange(len(self._balls))
      
    except ValueError:
      raise LookupError('pick from empty BingoCage')
      
    # Otherwise the randomly selected item is popped from self._balls.  
    return self._balls.pop(position)
  
  def loaded(self):
    # Override loaded to avoid calling inspect (as Tombola.loaded does in
    # Example 11-9). We can make it faster by working with self._balls directly—
    # no need to build a whole sorted tuple.
    return bool(self._balls)
  
  def inspect(self):
    # Override inspect with one-liner.
    return tuple(sorted(self._balls))
  
  
      
    

### Remark

Example 11-13 illustrates an idiom worth mentioning: in \_\_init\_\_, self._balls stores
list(iterable) and not just a reference to iterable (i.e., we did not merely assign
iterable to self._balls). As mentioned before,13 this makes our LotteryBlower
flexible because the iterable argument may be any iterable type. At the same time, we
make sure to store its items in a list so we can pop items. And even if we always get
lists as the iterable argument, list(iterable) produces a copy of the argument. 
which is a good practice considering we will be removing items from it and the client
may not be expecting the list of items she provided to be changed.

## A Virtual Subclass of Tombola

In goose typing, we can register a class as a virtual subclass of an ABC, even if it does not
inherit from it. When doing so, we promise that the class faithfully implements the
interface defined in the ABC—and Python will believe us without checking. If we lie,
we’ll be caught by the usual runtime exceptions.


This is done by calling a register method on the ABC. The registered class then be‐
comes a virtual subclass of the ABC, and will be recognized as such by functions like
issubclass and isinstance, but it will not inherit any methods or attributes from the
ABC.

### Remark

Virtual subclasses do not inherit from their registered ABCs, and
are not checked for conformance to the ABC interface at any time,
not even when they are instantiated. It’s up to the subclass to actually implement all the methods needed to avoid runtime errors